#### Libraries

In [1]:
import torch
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling)

##### config

In [ ]:
MODEL_NAME = "ai-forever/rugpt3medium_based_on_gpt2"
CACHE_DIR = 'data/finetune_gpt/model_cache'
OUTPUT_DIR = 'data/finetune_gpt/fine_tuned_model'
MODEL_DIR = "./trained_model"

#### Data

In [3]:
data_full = pd.read_csv("/content/synthetic.csv")

In [4]:
data_full.shape

(14040, 4)

In [5]:
data_full['prompt'] = data_full.apply(lambda row: f"Категория: {row['category']} | Отзыв: {row['clean_text']} | Анализ: {row['synthetic']}", axis=1)

In [6]:
data_full.head(3)

,category,sentiment,clean_text,synthetic,prompt
0,Общественное питание,Негативная,Никому не советую заказывать доставку из этого...,Сильные стороны:\n- нет конкретики\n\nСлабые с...,Категория: Общественное питание | Отзыв: Ником...
1,Общественное питание,Негативная,"Ужасно. Нормального подъезда нет, постоянно 3 ...",Сильные стороны:\n- отсутствуют\n\nСлабые стор...,Категория: Общественное питание | Отзыв: Ужасн...
2,Общественное питание,Негативная,Суп лапша и пюре картофельное кислое и испорче...,Сильные стороны:\n- отсутствуют\n\nСлабые стор...,Категория: Общественное питание | Отзыв: Суп л...


In [7]:
train_data, val_data = train_test_split(data_full, test_size = 0.1, random_state = 42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)

In [ ]:
def tokenize_data(texts, batch_size=1000, max_length=512):
    # Создаем словарь для хранения токенизированных данных
    encodings = {'input_ids': [], 'attention_mask': [], 'labels': []}
    
    # Проходим по текстам батчами
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        # Токенизируем текущий батч текстов
        batch_encodings = tokenizer(batch_texts, truncation=True, padding='max_length', max_length=max_length)
        encodings['input_ids'].extend(batch_encodings['input_ids'])
        encodings['attention_mask'].extend(batch_encodings['attention_mask'])
        encodings['labels'].extend(batch_encodings['input_ids'])

    return encodings

# Преобразуем тексты в список
train_texts = train_data['prompt'].tolist()
val_texts = val_data['prompt'].tolist()

# Токенизируем
train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

# Создаем датасеты из токенизированных данных
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# Выводим количество записей
print(f"Обучающая выборка: {len(train_dataset)} записей")
print(f"Валидационная выборка: {len(val_dataset)} записей")

Обучающая выборка: 12636 записей
Валидационная выборка: 1404 записей


In [ ]:
# Загружаем модель для обучения
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)

# Настройки для обучения модели
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, 
    overwrite_output_dir=True,
    num_train_epochs=1,  # Количество эпох обучения
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4, 
    gradient_accumulation_steps=2,  # Шаги накопления градиента
    evaluation_strategy="epoch", 
    eval_steps=None,  # Оценка в конце каждой эпохи
    save_strategy="epoch", 
    save_steps=None,  # Сохранение в конце каждой эпохи
    save_total_limit=2, 
    learning_rate=5e-5,
    lr_scheduler_type="linear", 
    warmup_ratio=0.1,
    weight_decay=0.01,  # Коэффициент регуляризации
    fp16=True, 
    load_best_model_at_end=True,  # Загрузка лучшей модели в конце обучения
    metric_for_best_model="loss",
    greater_is_better=False, 
    logging_strategy="steps", 
    logging_steps=50,
    report_to="none", 
)

# Создаем коллатор данных 
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Создаем объект Trainer для обучения модели
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator, 
)

# Запуск 
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,2.056300,1.996687


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1579, training_loss=2.3727000202084736, metrics={'train_runtime': 2002.9292, 'train_samples_per_second': 6.309, 'train_steps_per_second': 0.788, 'total_flos': 1.1731347173277696e+16, 'train_loss': 2.3727000202084736, 'epoch': 0.9996834441278886})

In [11]:
trainer.evaluate()

model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [12]:
MODEL_DIR = "./trained_model"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR)

model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
def generate_review(category, review, prompt_text="", max_length=256, min_length=64, num_return_sequences=1, temperature=0.4, top_p=0.50, top_k=30):
    input_prompt = f"Категория: {category} | Отзыв: {review}"
    inputs = tokenizer.encode(input_prompt, return_tensors="pt")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            do_sample=True,
            max_length=max_length,
            min_length=min_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=2,
            top_p=top_p,
            temperature=temperature,
            top_k=top_k,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_reviews = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return generated_reviews

category = "Общественное питание"
review = "Прекрасное кафе с вкусными завтраками до 16.00 и не только. Заказывала завтрак с кофе омлетом- заплатила 350 руб. Принесли омлет с хлебушком, овощами и капучино с моим портретом, я очень была удивлена и обрадована. Красивая посуда, замечательная подача блюда, приятная атмосфера. Рекомендую"
prompt_text = ""

output = generate_review(category, review, prompt_text)

for i, review in enumerate(output, 1):
    print(f"Отзыв {i}:\n{review}\n")

Отзыв 1:
Категория: Общественное питание | Отзыв: Прекрасное кафе с вкусными завтраками до 16.00 и не только. Заказывала завтрак с кофе омлетом- заплатила 350 руб. Принесли омлет с хлебушком, овощами и капучино с моим портретом, я очень была удивлена и обрадована. Красивая посуда, замечательная подача блюда, приятная атмосфера. Рекомендую к посещению. Вкусный кофе, очень вкусный, но не очень дорогой. Очень вкусно, особенно с молоком. Напитки тоже очень вкусные, все свежее. Персонал вежливый, всегда подскажут, помогут. Но, к сожалению, не всегда есть возможность быстро найти столик, поэтому лучше бронировать заранее. А так, в целом, кафе очень уютное, чисто, уютно, красиво. Есть парковка, где можно припарковаться. | Анализ: Сильные стороны:
1. Качество завтраков (до 16:00) и напитков (чай, кофе).
2. Внешний вид посуды (красивая, удобная посуда). 
3. Приятная атмосфера (упоминание об уютной обстановке).  
4. Доступность кофе и чая.
5. Быстрое обслуживание (заказ был сделан до

